# Notebook 3/4: Data cleaning (Repository and geographical coordinates)
***

# Table of contents
1. [Purpose of this notebook](#1)
2. [Repository of all LDAs, ZDLs and ZDEs](#2)
    1. [Presentation](#2A)
    1. [Exploration](#2B)
    2. [Conclusion](#2C)  
3. [Positions of all rail stations, by ZDL](#3)
    1. [Presentation](#3A)
    2. [Exploration](#3B)
    3. [Cleaning](#3C)
        1. [Filter metro stations](#3Ca)
        2. [Match station names with other dataframes](#3Cb)
        3. [Rename & filter columns](#3Cd)
        4. [Create LAT (latitude) and LONG (longitude) columns](#3Ce)
    4. [Export to csv](#3D)   
4. [Map of Paris metro network](#4)

# 1. Purpose of this notebook <a name="1"></a>

This notebook follows the exploration of the number of checkins and hourly profiles per metro station for the year 2018 (see notebook 2/4). For more general information about the data, please refer to notebook 1/4.

In order to enrich our understanding of the data at hand and to produce maps as part of our future analysis, this notebook will look into the following datasets:
- Repository of all LDAs, ZDLs and ZDEs
- Geographical data of all rail stations, by ZDL

In [51]:
# Import libraries

import numpy as np
import pandas as pd

# 2. Repository of all LDAs, ZDLs and ZDEs <a name="2"></a>

## 2.A. Presentation <a name="2A"></a>

The STIF provides numerous datasets with data about lines and stops.

[This dataset](https://opendata.stif.info/explore/dataset/referentiel-arret-tc-idf/information/) gives all the IDs for LDAs, ZDLs and ZDEs, by ZDE.

This page also includes documentation about this repository.

Note that the STIF provides a similar repository of all transport lines.

The datasets containing the number of checkins and hourly profiles for 2018 only include the LDA IDs and their related commercial name. This dataset can be useful to check if a STOP_NAME is valid, or to fill missing values in the ID_REFA_LDA column, by matching the names in the STOP_NAME column with those of the LDA_NOM column. It may also allow us to cross analyze datasets which do not include the same IDs.

#### Identification codes (see notebook 1/4):

The STIF assigns 3 levels of identification to each stop:

- LDA (Lieu D'Arrêt): Designates a place where vehicules from different lines can stop.
- ZDL (Zone De Lieu): Designates an area within an LDA that regroups several ZDEs with the same operating name.
- ZDE (Zone D'Embarquement): Designates a precise spot where people can get in and out of a vehicule (ex: metro platform).

The STIF also allocates internal codes to identify carriers (CODE_STIF_TRNS), network types (CODE_STIF_RES), and stops (CODE_STIF_ARRET).

Although I could not find the exhaustive list of codes and their meanings, I've identified that the CODE_STIF_RES 110 corresponds to the metro network in Paris. Our analysis focuses on Parisian metro stations.

## 2.B. Exploration <a name="2B"></a>

In [52]:
# Import libraries

import numpy as np
import pandas as pd

In [53]:
# Read file

ref_stops = pd.read_csv('../../datasets/referentiel-arret-tc-idf.csv', sep=';')

In [54]:
# Explore dataframe

ref_stops.head()

,ZDEr_ID_REF_A,ZDEr_NOM,ZDEr_ID_TYPE_ARRET,ZDEr_LIBELLE_TYPE_ARRET,ZDEr_X_Y,ZDLr_ID_REF_A,ZDLr_NOM,ZDLr_ID_TYPE_ARRET,ZDLr_LIBELLE_TYPE_ARRET,LDA_ID_REF_A,LDA_NOM,LDA_ID_TYPE_ARRET,LDA_LIBELLE_TYPE_ARRET
0,36757,Saint-Exupéry,5,Arrêt de bus,"640837,6863829",50507,Saint-Exupéry,5,Arrêt de bus,70831,Saint-Exupéry,5,Arrêt de bus
1,39355,La Paix,5,Arrêt de bus,"641034,6863697",50508,La Paix,5,Arrêt de bus,70820,La Paix,5,Arrêt de bus
2,39358,Victorien Sardou,5,Arrêt de bus,"641128,6863107",50510,Victorien Sardou,5,Arrêt de bus,70796,Victorien Sardou,5,Arrêt de bus
3,27653,Aristide Briand,5,Arrêt de bus,"658849,6869841",50518,Aristide Briand / Centre Culturel,5,Arrêt de bus,72619,Aristide Briand / Centre Culturel,5,Arrêt de bus
4,19289,Bois d'Amour,5,Arrêt de bus,"660488.75,6868398.5",50520,Jean Jaurès / Bois d'Amour,5,Arrêt de bus,72527,Jean Jaurès / Bois d'Amour,5,Arrêt de bus


This dataset does not include geographical data. The ZDEr_X_Y column looks like it contains coordinates, but these values do not correspond to the latitude and longitude of this region. This dataset does not contain the CODE_STIF_RES either, which we use in the other datasets to filter Parisian metro stations.

In [55]:
ref_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39925 entries, 0 to 39924
Data columns (total 13 columns):
ZDEr_ID_REF_A              39925 non-null int64
ZDEr_NOM                   39925 non-null object
ZDEr_ID_TYPE_ARRET         39925 non-null int64
ZDEr_LIBELLE_TYPE_ARRET    39925 non-null object
ZDEr_X_Y                   39925 non-null object
ZDLr_ID_REF_A              39925 non-null int64
ZDLr_NOM                   39925 non-null object
ZDLr_ID_TYPE_ARRET         39925 non-null int64
ZDLr_LIBELLE_TYPE_ARRET    39925 non-null object
LDA_ID_REF_A               39925 non-null int64
LDA_NOM                    39925 non-null object
LDA_ID_TYPE_ARRET          39925 non-null int64
LDA_LIBELLE_TYPE_ARRET     39925 non-null object
dtypes: int64(6), object(7)
memory usage: 4.0+ MB


In [56]:
ref_stops.nunique()

ZDEr_ID_REF_A              39925
ZDEr_NOM                   14500
ZDEr_ID_TYPE_ARRET             4
ZDEr_LIBELLE_TYPE_ARRET        4
ZDEr_X_Y                   39763
ZDLr_ID_REF_A              18412
ZDLr_NOM                   13001
ZDLr_ID_TYPE_ARRET             4
ZDLr_LIBELLE_TYPE_ARRET        4
LDA_ID_REF_A               15361
LDA_NOM                    11379
LDA_ID_TYPE_ARRET              4
LDA_LIBELLE_TYPE_ARRET         4
dtype: int64

Each row corresponds to a unique ZDE ID, but several ZDE IDs can share the same commercial name. There are no missing values.

Data types are consistent. There appears to be no need to clean this dataset, but rather to explore it a little bit further.

There seem to be 4 types of stops, whether accross ZDEs, ZDLs or LDAs. Let's look into it.

In [57]:
ref_stops['ZDEr_ID_TYPE_ARRET'].value_counts()

5    36695
1     2049
2      768
6      413
Name: ZDEr_ID_TYPE_ARRET, dtype: int64

In [58]:
ref_stops['ZDEr_LIBELLE_TYPE_ARRET'].value_counts()

Arrêt de bus            36695
Station ferrée / Val     2049
Station de métro          768
Arrêt de tram             413
Name: ZDEr_LIBELLE_TYPE_ARRET, dtype: int64

#### Stop types (see notebook 1/4)

There are 4 types of stops:

- 5: Arrêt de bus (=bus stop)
- 1: Station ferrée / Val (=rail station)
- 2: Station de métro (=metro station)
- 6: Arrêt de tram (=tram stop)

In [59]:
ZDE_val_counts = ref_stops['ZDEr_ID_TYPE_ARRET'].value_counts()
ZDL_val_counts = ref_stops['ZDLr_ID_TYPE_ARRET'].value_counts()
LDA_val_counts = ref_stops['LDA_ID_TYPE_ARRET'].value_counts()

ZDE_counts = pd.DataFrame(data=ZDE_val_counts.values, index=ZDE_val_counts.index, columns = ['ZDE'])
ZDL_counts = pd.DataFrame(data=ZDL_val_counts.values, index=ZDL_val_counts.index, columns = ['ZDL'])
LDA_counts = pd.DataFrame(data=LDA_val_counts.values, index=LDA_val_counts.index, columns = ['LDA'])

In [60]:
ref_stops['LDA_ID_TYPE_ARRET'].value_counts().values

array([31819,  5065,  2208,   833])

In [61]:
ZDE_counts

,ZDE
5,36695
1,2049
2,768
6,413


In [62]:
ids_counts = pd.concat([ZDE_counts, ZDL_counts, LDA_counts], axis=1)

ids_counts 

,ZDE,ZDL,LDA
5,36695,36691,31819
1,2049,2059,5065
2,768,762,2208
6,413,413,833


The number of value counts per type of stops is not the same accross all of the ID columns.

In [63]:
gbo_ids = ref_stops.groupby(['LDA_ID_TYPE_ARRET', 'ZDLr_ID_TYPE_ARRET', 'ZDEr_ID_TYPE_ARRET'])

In [64]:
gp_ids_counts = pd.DataFrame(gbo_ids.count()['LDA_ID_REF_A'])

gp_ids_counts

LDA_ID_REF_A
LDA_ID_TYPE_ARRET ZDLr_ID_TYPE_ARRET ZDEr_ID_TYPE_ARRET              
1                 1                  1                           2049
                                     2                              6
                                     5                              4
                  2                  2                             78
                  5                  5                           2884
                  6                  6                             44
2                 2                  2                            684
                  5                  5                           1471
                  6                  6                             53
5                 5                  5                          31819
6                 5                  5                            517
                  6                  6                            316

A single LDA which is categorised as train station, for instance, might include several ZDLs among which some are train stations, others are metro stations, and so on (see gp_ids_counts dataframe). This explains why the number of value counts per type of stops is not the same accross all of the ID columns.

## 2.C. Conclusion <a name="2C"></a>

This dataset does not need to be cleaned since we will use it in a fairly superficial way. 

There are 3 identification levels: LDA, ZDL and ZDE. Each LDA can contain several ZDLS that themselves contain a number of ZDEs. Each of these IDs has 4 stop types: metro, rail, bus and tram.

 # 3. Positions of all rail stations, by ZDL <a name="3"></a>

## 3.A. Presentation <a name="3A"></a>

This dataset provides geographic information about all rail, metro, tram and funicular stops in Ile de France:
- https://opendata.stif.info/explore/dataset/emplacement-des-gares-idf-data-generalisee/information/

Note that the STIF provides similar data grouped by transport line.

This dataset should be particularly useful in order to produce maps.

As a reminder, here are the steps we identified in order to clean it:

> Filter metro stations

> Match station names of geo_stops with that of other dataframes

> Update column names and drop useless columns

> Create latitude and longitude columns
 
> Export to .csv

In [65]:
# Read file

geo_stops = pd.read_csv('../../datasets/emplacement-des-gares-idf-data-generalisee.csv', sep=';')

## 3.B. Exploration <a name="3B"></a>

In [66]:
# Explore dataframe

geo_stops.head()

,Geo Point,Geo Shape,id_ref_zdl,nom_long,label,idrefliga,idrefligc,res_com,mode_,fer,...,terrer,termetro,tertram,ternavette,terval,exploitant,principal,idf,x,y
0,"48.8463569889, 2.41947990037","{""type"": ""Point"", ""coordinates"": [2.4194799003...",47247,SAINT-MANDE,Saint-Mandé,A01534,C01371,M1,Metro,0,...,0,0,0,0,0,RATP,0,1,657397.0779,6.860858e+06
1,"48.8662858046, 2.32294341224","{""type"": ""Point"", ""coordinates"": [2.3229434122...",45676,CONCORDE,Concorde,A01534 / A01541 / A01545,C01371 / C01378 / C01382,M1 / M8 / M12,Metro,0,...,0,0,0,0,0,RATP,0,1,650331.6676,6.863130e+06
2,"48.8828686476, 2.34413063372","{""type"": ""Point"", ""coordinates"": [2.3441306337...",42210,ANVERS,Anvers,A01535,C01372,M2,Metro,0,...,0,0,0,0,0,RATP,0,1,651901.2249,6.864961e+06
3,"48.9078125468, 2.45435282652","{""type"": ""Point"", ""coordinates"": [2.4543528265...",47334,JEAN ROSTAND,Jean Rostand,A01191,C01389,T1,Tramway,0,...,0,0,0,0,0,RATP,0,1,660003.4947,6.867673e+06
4,"48.8930946286, 2.48791098763","{""type"": ""Point"", ""coordinates"": [2.4879109876...",44603,LA REMISE A JORELLE,La Remise à Jorelle,A01761,C01843,T4,Tramway,0,...,0,0,0,0,0,SNCF,0,1,662452.6934,6.866020e+06


In [67]:
geo_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 28 columns):
Geo Point     923 non-null object
Geo Shape     923 non-null object
id_ref_zdl    923 non-null int64
nom_long      923 non-null object
label         923 non-null object
idrefliga     890 non-null object
idrefligc     847 non-null object
res_com       923 non-null object
mode_         923 non-null object
fer           923 non-null int64
train         923 non-null int64
rer           923 non-null int64
metro         923 non-null int64
tramway       923 non-null int64
navette       923 non-null int64
val           923 non-null int64
terfer        923 non-null object
tertrain      923 non-null object
terrer        923 non-null object
termetro      923 non-null object
tertram       923 non-null object
ternavette    923 non-null object
terval        923 non-null object
exploitant    923 non-null object
principal     923 non-null int64
idf           923 non-null int64
x             923 non

In [68]:
geo_stops['id_ref_zdl'].nunique()

923

This dataset contains the geo coordinates of all stations of the rail network in Ile de France, by ZDL. It contains 923 unique ZDLs (corresponding to the total number of rows). As we saw earlier, there are multiple ZDLs for one LDA.

The datasets we are analyzing (metro_nb and metro_hp) only contain LDAs (301 unique values).

In order to match the data from metro_nb and metro_hp and that of geo_stops, we need to filter geo_stops by metro stations and to make sure that we can match their LDAs.

## 3.C. Cleaning <a name="3C"></a>

### 3.C.a. Filter metro stations <a name="3Ca"></a>

In [69]:
# Array of all the mode types

geo_stops['mode_'].unique()

array(['Metro', 'Tramway', 'Train', 'Navette', 'RER', 'Metro / Tramway',
       'Train / Tramway', 'Train / RER', 'RER / Tramway', 'RER / Metro',
       'Train / Metro', 'RER / Navette', 'Train / RER / Tramway',
       'Train / RER / Metro', 'Navette / Tramway',
       'Train / RER / Metro / Tramway', 'RER / Metro / Tramway'],
      dtype=object)

In [70]:
# List of mode types containing the word metro

metro = geo_stops.loc[geo_stops['mode_'].str.contains('Metro'),'mode_'].unique().tolist()

metro

['Metro',
 'Metro / Tramway',
 'RER / Metro',
 'Train / Metro',
 'Train / RER / Metro',
 'Train / RER / Metro / Tramway',
 'RER / Metro / Tramway']

In [71]:
# geo_stops filtered by metro (mode_ contains the word 'metro')

geo_metro = geo_stops.loc[geo_stops['mode_'].isin(metro)].copy()

In [72]:
# Number of ZDL IDs (rows) in geo_metro

len(geo_metro['id_ref_zdl'])

302

In [73]:
# List of ZDL IDs for metro stations

zdl_metro = geo_metro['id_ref_zdl'].unique().tolist()

# Number of ZDEs (rows of ref_stops) matching zdl_metro

len(ref_stops[ref_stops['ZDLr_ID_REF_A'].isin(zdl_metro)])

806

This is good news, when we filtered geo_stops, we obtained a dataframe that has 302 rows, which is very close to the unique number of ID_REFA_LDA in metro_nb and metro_hp (301). 

Let's find the extra value in geo_metro, and see if the names ('nom_long') in geo_metro match those of metro_nb ('STOP_NAME')

### 3.C.b Match station names in geo_metro with names in metro_nb and metro_hp <a name="3Cb"></a>

In [74]:
metro_nb = pd.read_csv('../../datasets/metro_nb.csv')

In [75]:
sorted_metro_nb_names = np.sort(metro_nb['STOP_NAME'].unique())

In [76]:
sorted_geo_names = np.sort(geo_metro['nom_long'].unique())

In [77]:
# Names in geo_metro that are not in metro_nb (& metro_hp)

for name in sorted_geo_names:
    if name in sorted_metro_nb_names:
        continue
    else:
        print(name)

CLUNY-LA-SORBONNE
CRETEIL-POINTE DU LAC
FRONT POPULAIRE
LES COURTILLES-ASNIERES-GENNEVILLIERS
MAIRIE DE MONTROUGE
MALAKOFF-RUE ÉTIENNE DOLET
SAINT-MANDE


In [78]:
# Names in metro_nb (& metro_hp) that are not in geo_metro

for name in sorted_metro_nb_names:
    if name in sorted_geo_names:
        continue
    else:
        print(name)

CRETEIL-P. LAC
FR. POPULAIRE
LES COURTILLES
M. MONTROUGE
MALAKOFF-RUE ETIENNE DOLET
SAINT-MANDE-TOURELLE


We see that the extra row in geo_metro corresponds to CLUNY-LA-SORBONNE. After checking the original data of metro_nb and metro_hp, it looks like CLUNY-LA-SORBONNE is missing, no record of the number of checkins for that metro station was provided. We will have to do without it.

Some other names do not match, but we can replace them so that they match.

In [79]:
# Dictionary of names to update:

names = {'CRETEIL-POINTE DU LAC':'CRETEIL-P. LAC',
         'FRONT POPULAIRE':'FR. POPULAIRE',
         'LES COURTILLES-ASNIERES-GENNEVILLIERS':'LES COURTILLES',
         'MAIRIE DE MONTROUGE':'M. MONTROUGE',
         'MALAKOFF-RUE ÉTIENNE DOLET':'MALAKOFF-RUE ETIENNE DOLET',
         'SAINT-MANDE':'SAINT-MANDE-TOURELLE'}

In [80]:
# Replace names in geo_metro

geo_metro['nom_long'] = geo_metro['nom_long'].replace(names)

In [81]:
geo_metro.columns

Index(['Geo Point', 'Geo Shape', 'id_ref_zdl', 'nom_long', 'label',
       'idrefliga', 'idrefligc', 'res_com', 'mode_', 'fer', 'train', 'rer',
       'metro', 'tramway', 'navette', 'val', 'terfer', 'tertrain', 'terrer',
       'termetro', 'tertram', 'ternavette', 'terval', 'exploitant',
       'principal', 'idf', 'x', 'y'],
      dtype='object')

### 3.C.c. Rename & filter columns <a name="3Cc"></a>

In [82]:
# Dictionary with old names (keys) and new names (values)
col_names = {'Geo Point':'LAT_LONG', 
             'id_ref_zdl':'ID_ZDL', 
             'nom_long':'STOP_NAME', 
             'res_com':'LINES'}

# Rename columns
geo_metro.rename(columns=col_names, inplace=True)

# List of columns to drop
col_todrop = ['Geo Shape', 'label', 'idrefliga', 'idrefligc', 'mode_', 'fer', 'train', 'rer',
              'metro', 'tramway', 'navette', 'val', 'terfer', 'tertrain', 'terrer',
              'termetro', 'tertram', 'ternavette', 'terval', 'exploitant','principal', 'idf', 'x', 'y']

# Drop columns
geo_metro.drop(columns=col_todrop, inplace=True)

In [83]:
geo_metro.head()

,LAT_LONG,ID_ZDL,STOP_NAME,LINES
0,"48.8463569889, 2.41947990037",47247,SAINT-MANDE-TOURELLE,M1
1,"48.8662858046, 2.32294341224",45676,CONCORDE,M1 / M8 / M12
2,"48.8828686476, 2.34413063372",42210,ANVERS,M2
11,"48.8822269008, 2.32127492677",44662,ROME,M2
12,"48.8804494561, 2.30945116799",44146,MONCEAU,M2


In [84]:
geo_metro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302 entries, 0 to 891
Data columns (total 4 columns):
LAT_LONG     302 non-null object
ID_ZDL       302 non-null int64
STOP_NAME    302 non-null object
LINES        302 non-null object
dtypes: int64(1), object(3)
memory usage: 11.8+ KB


###  3.C.d. Create LAT (latitude) and LONG (longitude) columns <a name="3Cd"></a>

In [85]:
geo_metro["LAT"] = geo_metro["LAT_LONG"].str.split(",").str.get(0)
geo_metro["LONG"] = geo_metro["LAT_LONG"].str.split(",").str.get(1)

In [86]:
geo_metro.head()

,LAT_LONG,ID_ZDL,STOP_NAME,LINES,LAT,LONG
0,"48.8463569889, 2.41947990037",47247,SAINT-MANDE-TOURELLE,M1,48.8463569889,2.41947990037
1,"48.8662858046, 2.32294341224",45676,CONCORDE,M1 / M8 / M12,48.8662858046,2.32294341224
2,"48.8828686476, 2.34413063372",42210,ANVERS,M2,48.8828686476,2.34413063372
11,"48.8822269008, 2.32127492677",44662,ROME,M2,48.8822269008,2.32127492677
12,"48.8804494561, 2.30945116799",44146,MONCEAU,M2,48.8804494561,2.30945116799


## 3.D. Export to .csv <a name="3D"></a>

In [87]:
geo_metro.to_csv('../../datasets/geo_metro.csv', index=None)

# 4. Map of Paris metro network <a name="4"></a>

In [88]:
from ipywidgets import HTML
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Icon, Popup, Heatmap

In [89]:
center=(48.855115,2.352448)

m = Map(
    center=center,
    zoom=12
)


for x in geo_metro.iterrows():
    lat = x[1].LAT
    long = x[1].LONG
    
    n = x[1].STOP_NAME
    
    message = HTML()
    
    message.value = n
    
    circle_marker = CircleMarker(location=(lat, long), draggable=False)
    circle_marker.radius = 6
    circle_marker.stroke = False
    circle_marker.color = '#000951'
    circle_marker.opacity = 0.7
    circle_marker.fill_color = '#000951'
    circle_marker.fill_opacity = 0.7
    m.add_layer(circle_marker);
    
    circle_marker.popup = message

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[48.855115, 2.352448], layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8463569889', ' 2.41947990037'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='SAINT-MANDE-TOURELLE'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8662858046', ' 2.32294341224'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='CONCORDE'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8828686476', ' 2.34413063372'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='ANVERS'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8822269008', ' 2.32127492677'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='ROME'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8804494561', ' 2.30945116799'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='MONCEAU'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.862949833', ' 2.387434951'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='PERE LACHAISE'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8738067339', ' 2.32699985888'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='HAVRE-CAUMARTIN'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8828919175', ' 2.30905226455'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='MALESHERBES'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8887060198', ' 2.28808363305'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='LOUISE MICHEL'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.8837344657', ' 2.34963614827'], opacity=0.7, options=['class_name', 'color', 'dash_array', 'fill', 'fill_color', 'fill_opacity', 'line_cap', 'line_join', 'opacity', 'pointer_events', 'radius', 'stroke', 'weight'], popup=HTML(value='BARBES-ROCHECHOUART'), radius=6, stroke=False), CircleMarker(color='#000951', fill_color='#000951', fill_opacity=0.7, location=['48.